In [1]:
#data wrangling tools
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np

#import models
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#stats for ensembling

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

#Preprocessing
from sklearn.preprocessing import OrdinalEncoder

#Ensembling
from scipy import stats as st

In [2]:
def fold(fold_list,K):
    stack=np.column_stack((fold_list[i] for i in range(K)))
    mode=st.mode(stack,axis=1)
    return mode[0]

In [3]:
#Category encoder
def cat_encoder(X_train, X_test, cat_cols):
    encoder = OrdinalEncoder()
    train_encoder = encoder.fit_transform(X_train[cat_cols]).astype(int)
    test_encoder = encoder.transform(X_test[cat_cols]).astype(int)
    for col in cat_cols:
        X_train[col] = train_encoder[:, cat_cols.index(col)]
        X_test[col] = test_encoder[:, cat_cols.index(col)]
    encoder_cols = cat_cols
    return X_train, X_test, encoder_cols

In [4]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.feature_selection import VarianceThreshold

#Implementingthe Transformer class
class low_var(TransformerMixin):
    def __init__(self,threshold=0.3):
        self.threshold=threshold
    def fit(self,X,y=None):
        col_vars=X.var()
        self.col_to_drop=col_vars[col_vars<self.threshold].index
        return self
    def transform(self,X):
        assert self.col_to_drop is not None, 'Drop_col error, must be fitted before predict'
        X.drop(self.col_to_drop, axis=1, inplace=True)
        return X

In [5]:
#Reading the dataset
FILEPATH="/kaggle/input/Analytic-olympiad"
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

#Defining the features
target_col=df_train.columns.to_list()[-2]
cat_cols=df_train.select_dtypes(include="object").columns.to_list()
num_cols=df_train.select_dtypes(include=["int64","float64"]).columns.to_list()[:-2]

#Definging the training and testing dataset
X_train = df_train.drop([f'{target_col}'],axis=1).reset_index(drop=True)
y_train = df_train[f'{target_col}'].reset_index(drop=True)
X_test=df_test.copy()

In [6]:
# Drop_col
encoded_features = ['encoded_payment_' + str(i) for i in range(25)]
for col in encoded_features:
    X_train.drop([col], axis=1, inplace=True)
    X_test.drop([col],axis =1, inplace =True)
drop_cols = ['customer_id', 'firstname', 'lastname']
X_train.drop(drop_cols, axis=1, inplace=True)
X_test.drop(drop_cols, axis=1, inplace=True)

#Reassigning cat_cols
cat_cols=X_train.select_dtypes(include="object").columns.to_list()
num_cols=X_train.select_dtypes(include=["int64","float64"]).columns.to_list()[:-2]

#Removing columns with low variance
var_col=low_var(1)
X_train=pd.concat([var_col.fit_transform(X_train[num_cols]),X_train[cat_cols]],axis=1)
X_test=pd.concat([var_col.transform(X_test[num_cols]),X_test[cat_cols]],axis=1)

# category_encoders
X_train, X_test, cat_cols = cat_encoder(X_train, X_test, cat_cols)

In [7]:
class Splitter:
    def __init__(self, kfold=True, n_splits=5, cat_df=pd.DataFrame()):
        self.n_splits = n_splits
        self.kfold = kfold
        self.cat_df = cat_df

    def split_data(self, X, y, random_state_list):
        if self.kfold == 'skf':
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, self.cat_df):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        elif self.kfold:
            for random_state in random_state_list:
                kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val
        else:
            raise ValueError(f"Invalid kfold: Must be True")

In [8]:
class Classifier:
    def __init__(self):
        self.models = self._define_model()
        self.models_name = list(self._define_model().keys())
        self.len_models = len(self.models)
        
    def _define_model(self):
        lgbm_params={
        }
        
        xgb_params =  {
           
        }  
        
        rf_params =  {
            
        }
        
        cat_params = {

        }
        
       
        gdbt_params =  {
        }
        
        models = {
            'xgb': XGBClassifier(**xgb_params),
            'gdbt':GradientBoostingClassifier(**gdbt_params),
            'cat': CatBoostClassifier(**cat_params),
            'rf' : RandomForestClassifier(**rf_params),
            'lgbm':LGBMClassifier(**lgbm_params)
        }
        
        return models

In [9]:
kfold = True
n_splits = 5
random_state_list = [42] 
classifier = Classifier()
splitter = Splitter(kfold=kfold, n_splits=n_splits)
score_dict = dict(zip(classifier.models_name, [[] for _ in range(classifier.len_models)]))
test_predss = np.zeros((X_test.shape[0]))
final=[]
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
    clf=Classifier()
    models=clf.models
    test_preds = []
    oof_preds = []
    
    for name, model in models.items():
        model.fit(X_train_, y_train_)
        y_val_pred =model.predict(X_val).reshape(-1)
        test_pred = model.predict(X_test).reshape(-1)
        
        
        score= [roc_auc_score(y_val, y_val_pred)]
        score_dict[name].append(np.mean(score))
        
        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)

    final.append(fold(test_preds,classifier.len_models))
        
test_predss=fold(final,n_splits)    

Learning rate set to 0.186788
0:	learn: 0.1543461	total: 291ms	remaining: 4m 50s
1:	learn: 0.0367624	total: 383ms	remaining: 3m 11s
2:	learn: 0.0112693	total: 474ms	remaining: 2m 37s
3:	learn: 0.0045336	total: 555ms	remaining: 2m 18s
4:	learn: 0.0024677	total: 637ms	remaining: 2m 6s
5:	learn: 0.0015083	total: 716ms	remaining: 1m 58s
6:	learn: 0.0012410	total: 790ms	remaining: 1m 52s
7:	learn: 0.0010813	total: 862ms	remaining: 1m 46s
8:	learn: 0.0009898	total: 936ms	remaining: 1m 43s
9:	learn: 0.0008754	total: 1000ms	remaining: 1m 38s
10:	learn: 0.0007860	total: 1.07s	remaining: 1m 36s
11:	learn: 0.0007393	total: 1.13s	remaining: 1m 33s
12:	learn: 0.0007082	total: 1.21s	remaining: 1m 32s
13:	learn: 0.0006638	total: 1.29s	remaining: 1m 30s
14:	learn: 0.0006452	total: 1.36s	remaining: 1m 29s
15:	learn: 0.0006064	total: 1.44s	remaining: 1m 28s
16:	learn: 0.0005965	total: 1.52s	remaining: 1m 28s
17:	learn: 0.0005790	total: 1.6s	remaining: 1m 27s
18:	learn: 0.0005726	total: 1.67s	remaining: 

162:	learn: 0.0003402	total: 11.2s	remaining: 57.6s
163:	learn: 0.0003402	total: 11.3s	remaining: 57.5s
164:	learn: 0.0003402	total: 11.3s	remaining: 57.4s
165:	learn: 0.0003402	total: 11.4s	remaining: 57.3s
166:	learn: 0.0003402	total: 11.5s	remaining: 57.2s
167:	learn: 0.0003402	total: 11.5s	remaining: 57.1s
168:	learn: 0.0003402	total: 11.6s	remaining: 57.1s
169:	learn: 0.0003402	total: 11.7s	remaining: 57s
170:	learn: 0.0003402	total: 11.7s	remaining: 56.9s
171:	learn: 0.0003402	total: 11.8s	remaining: 56.8s
172:	learn: 0.0003402	total: 11.9s	remaining: 56.7s
173:	learn: 0.0003402	total: 11.9s	remaining: 56.7s
174:	learn: 0.0003402	total: 12s	remaining: 56.6s
175:	learn: 0.0003402	total: 12.1s	remaining: 56.5s
176:	learn: 0.0003402	total: 12.1s	remaining: 56.5s
177:	learn: 0.0003402	total: 12.2s	remaining: 56.4s
178:	learn: 0.0003402	total: 12.3s	remaining: 56.3s
179:	learn: 0.0003402	total: 12.3s	remaining: 56.2s
180:	learn: 0.0003402	total: 12.4s	remaining: 56.1s
181:	learn: 0.00

321:	learn: 0.0003402	total: 21.7s	remaining: 45.7s
322:	learn: 0.0003402	total: 21.8s	remaining: 45.7s
323:	learn: 0.0003402	total: 21.9s	remaining: 45.6s
324:	learn: 0.0003402	total: 21.9s	remaining: 45.5s
325:	learn: 0.0003402	total: 22s	remaining: 45.5s
326:	learn: 0.0003402	total: 22s	remaining: 45.4s
327:	learn: 0.0003402	total: 22.1s	remaining: 45.3s
328:	learn: 0.0003402	total: 22.2s	remaining: 45.2s
329:	learn: 0.0003402	total: 22.2s	remaining: 45.2s
330:	learn: 0.0003402	total: 22.3s	remaining: 45.1s
331:	learn: 0.0003402	total: 22.4s	remaining: 45s
332:	learn: 0.0003402	total: 22.4s	remaining: 45s
333:	learn: 0.0003402	total: 22.5s	remaining: 44.9s
334:	learn: 0.0003402	total: 22.6s	remaining: 44.8s
335:	learn: 0.0003402	total: 22.6s	remaining: 44.7s
336:	learn: 0.0003402	total: 22.7s	remaining: 44.7s
337:	learn: 0.0003402	total: 22.8s	remaining: 44.6s
338:	learn: 0.0003402	total: 22.8s	remaining: 44.5s
339:	learn: 0.0003402	total: 22.9s	remaining: 44.5s
340:	learn: 0.000340

483:	learn: 0.0003402	total: 32.3s	remaining: 34.5s
484:	learn: 0.0003402	total: 32.4s	remaining: 34.4s
485:	learn: 0.0003402	total: 32.5s	remaining: 34.3s
486:	learn: 0.0003402	total: 32.5s	remaining: 34.3s
487:	learn: 0.0003402	total: 32.6s	remaining: 34.2s
488:	learn: 0.0003402	total: 32.7s	remaining: 34.1s
489:	learn: 0.0003402	total: 32.7s	remaining: 34.1s
490:	learn: 0.0003402	total: 32.8s	remaining: 34s
491:	learn: 0.0003402	total: 32.9s	remaining: 33.9s
492:	learn: 0.0003402	total: 32.9s	remaining: 33.9s
493:	learn: 0.0003402	total: 33s	remaining: 33.8s
494:	learn: 0.0003402	total: 33.1s	remaining: 33.7s
495:	learn: 0.0003402	total: 33.1s	remaining: 33.7s
496:	learn: 0.0003402	total: 33.2s	remaining: 33.6s
497:	learn: 0.0003402	total: 33.3s	remaining: 33.5s
498:	learn: 0.0003402	total: 33.3s	remaining: 33.5s
499:	learn: 0.0003402	total: 33.4s	remaining: 33.4s
500:	learn: 0.0003402	total: 33.5s	remaining: 33.3s
501:	learn: 0.0003402	total: 33.5s	remaining: 33.3s
502:	learn: 0.00

645:	learn: 0.0003402	total: 43s	remaining: 23.5s
646:	learn: 0.0003402	total: 43s	remaining: 23.5s
647:	learn: 0.0003402	total: 43.1s	remaining: 23.4s
648:	learn: 0.0003402	total: 43.2s	remaining: 23.4s
649:	learn: 0.0003402	total: 43.2s	remaining: 23.3s
650:	learn: 0.0003402	total: 43.3s	remaining: 23.2s
651:	learn: 0.0003402	total: 43.4s	remaining: 23.2s
652:	learn: 0.0003402	total: 43.4s	remaining: 23.1s
653:	learn: 0.0003402	total: 43.5s	remaining: 23s
654:	learn: 0.0003402	total: 43.6s	remaining: 22.9s
655:	learn: 0.0003402	total: 43.6s	remaining: 22.9s
656:	learn: 0.0003402	total: 43.7s	remaining: 22.8s
657:	learn: 0.0003402	total: 43.8s	remaining: 22.7s
658:	learn: 0.0003402	total: 43.8s	remaining: 22.7s
659:	learn: 0.0003402	total: 43.9s	remaining: 22.6s
660:	learn: 0.0003402	total: 44s	remaining: 22.5s
661:	learn: 0.0003402	total: 44s	remaining: 22.5s
662:	learn: 0.0003402	total: 44.1s	remaining: 22.4s
663:	learn: 0.0003402	total: 44.2s	remaining: 22.3s
664:	learn: 0.0003402	

...

478:	learn: 0.0002903	total: 29.8s	remaining: 32.4s
479:	learn: 0.0002903	total: 29.8s	remaining: 32.3s
480:	learn: 0.0002903	total: 29.9s	remaining: 32.2s
481:	learn: 0.0002903	total: 29.9s	remaining: 32.2s
482:	learn: 0.0002903	total: 30s	remaining: 32.1s
483:	learn: 0.0002903	total: 30.1s	remaining: 32.1s
484:	learn: 0.0002903	total: 30.1s	remaining: 32s
485:	learn: 0.0002903	total: 30.2s	remaining: 31.9s
486:	learn: 0.0002903	total: 30.3s	remaining: 31.9s
487:	learn: 0.0002903	total: 30.3s	remaining: 31.8s
488:	learn: 0.0002903	total: 30.4s	remaining: 31.7s
489:	learn: 0.0002903	total: 30.4s	remaining: 31.7s
490:	learn: 0.0002903	total: 30.5s	remaining: 31.6s
491:	learn: 0.0002903	total: 30.6s	remaining: 31.6s
492:	learn: 0.0002903	total: 30.6s	remaining: 31.5s
493:	learn: 0.0002903	total: 30.7s	remaining: 31.4s
494:	learn: 0.0002903	total: 30.7s	remaining: 31.4s
495:	learn: 0.0002903	total: 30.8s	remaining: 31.3s
496:	learn: 0.0002903	total: 30.9s	remaining: 31.2s
497:	learn: 0.00

637:	learn: 0.0002903	total: 39.4s	remaining: 22.4s
638:	learn: 0.0002903	total: 39.5s	remaining: 22.3s
639:	learn: 0.0002903	total: 39.5s	remaining: 22.2s
640:	learn: 0.0002903	total: 39.6s	remaining: 22.2s
641:	learn: 0.0002903	total: 39.6s	remaining: 22.1s
642:	learn: 0.0002903	total: 39.7s	remaining: 22s
643:	learn: 0.0002903	total: 39.8s	remaining: 22s
644:	learn: 0.0002903	total: 39.8s	remaining: 21.9s
645:	learn: 0.0002903	total: 39.9s	remaining: 21.9s
646:	learn: 0.0002903	total: 39.9s	remaining: 21.8s
647:	learn: 0.0002903	total: 40s	remaining: 21.7s
648:	learn: 0.0002903	total: 40.1s	remaining: 21.7s
649:	learn: 0.0002903	total: 40.1s	remaining: 21.6s
650:	learn: 0.0002903	total: 40.2s	remaining: 21.5s
651:	learn: 0.0002903	total: 40.3s	remaining: 21.5s
652:	learn: 0.0002903	total: 40.3s	remaining: 21.4s
653:	learn: 0.0002903	total: 40.4s	remaining: 21.4s
654:	learn: 0.0002903	total: 40.4s	remaining: 21.3s
655:	learn: 0.0002903	total: 40.5s	remaining: 21.2s
656:	learn: 0.0002

798:	learn: 0.0002903	total: 49.2s	remaining: 12.4s
799:	learn: 0.0002903	total: 49.2s	remaining: 12.3s
800:	learn: 0.0002903	total: 49.3s	remaining: 12.2s
801:	learn: 0.0002903	total: 49.3s	remaining: 12.2s
802:	learn: 0.0002903	total: 49.4s	remaining: 12.1s
803:	learn: 0.0002903	total: 49.5s	remaining: 12.1s
804:	learn: 0.0002903	total: 49.5s	remaining: 12s
805:	learn: 0.0002903	total: 49.6s	remaining: 11.9s
806:	learn: 0.0002903	total: 49.6s	remaining: 11.9s
807:	learn: 0.0002903	total: 49.7s	remaining: 11.8s
808:	learn: 0.0002903	total: 49.8s	remaining: 11.7s
809:	learn: 0.0002903	total: 49.8s	remaining: 11.7s
810:	learn: 0.0002903	total: 49.9s	remaining: 11.6s
811:	learn: 0.0002903	total: 49.9s	remaining: 11.6s
812:	learn: 0.0002903	total: 50s	remaining: 11.5s
813:	learn: 0.0002903	total: 50.1s	remaining: 11.4s
814:	learn: 0.0002903	total: 50.1s	remaining: 11.4s
815:	learn: 0.0002903	total: 50.2s	remaining: 11.3s
816:	learn: 0.0002903	total: 50.2s	remaining: 11.3s
817:	learn: 0.00

959:	learn: 0.0002903	total: 58.9s	remaining: 2.45s
960:	learn: 0.0002903	total: 59s	remaining: 2.39s
961:	learn: 0.0002903	total: 59s	remaining: 2.33s
962:	learn: 0.0002903	total: 59.1s	remaining: 2.27s
963:	learn: 0.0002903	total: 59.1s	remaining: 2.21s
964:	learn: 0.0002903	total: 59.2s	remaining: 2.15s
965:	learn: 0.0002903	total: 59.3s	remaining: 2.08s
966:	learn: 0.0002903	total: 59.3s	remaining: 2.02s
967:	learn: 0.0002903	total: 59.4s	remaining: 1.96s
968:	learn: 0.0002903	total: 59.4s	remaining: 1.9s
969:	learn: 0.0002903	total: 59.5s	remaining: 1.84s
970:	learn: 0.0002903	total: 59.6s	remaining: 1.78s
971:	learn: 0.0002903	total: 59.6s	remaining: 1.72s
972:	learn: 0.0002903	total: 59.7s	remaining: 1.66s
973:	learn: 0.0002903	total: 59.8s	remaining: 1.59s
974:	learn: 0.0002903	total: 59.8s	remaining: 1.53s
975:	learn: 0.0002903	total: 59.9s	remaining: 1.47s
976:	learn: 0.0002903	total: 59.9s	remaining: 1.41s
977:	learn: 0.0002903	total: 60s	remaining: 1.35s
978:	learn: 0.00029

C:\Users\VaishakhRaveendran\AppData\Local\Temp\ipykernel_11248\1986525794.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  stack=np.column_stack((fold_list[i] for i in range(K)))


In [10]:
result=pd.read_csv('submission_trivial_base.csv')
result['primary_close_flag']=test_predss

In [13]:
score_dict

{'xgb': [0.9997202469957555,
  0.999662928802148,
  0.9997338808983546,
  0.999829620387286,
  0.9998280849944365],
 'gdbt': [0.9996156769707036,
  0.9996591934775927,
  0.9995035296623374,
  0.9996382048513605,
  0.9997245255765664],
 'cat': [0.9996969171532578,
  0.9996928924883971,
  0.9997131347636188,
  0.9997659264075732,
  0.9997732271765114],
 'rf': [0.9997879560318502,
  0.9996453547274229,
  0.999713760991019,
  0.9997659264075732,
  0.9998332682752461],
 'lgbm': [0.9958066702676417,
  0.9995307249905305,
  0.9997559678451318,
  0.9996695109542878,
  0.9995225900216358]}

In [11]:
result.to_csv('submission_XXPPP.csv',index=False)